In [1]:
import os
import json
from docx import Document
from io import StringIO, BytesIO
import re

import pandas as pd
import json
import spacy
from nltk.corpus import stopwords

from gensim.models import LdaModel
from gensim.models.wrappers import LdaMallet
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim import matutils, models
from gensim.models import CoherenceModel
import pyLDAvis.gensim

from docx import Document
from io import StringIO, BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 14, 'lines.linewidth': 3})

In [2]:
nlp = spacy.load("en_core_web_sm")
# stop_words = set(stopwords.words('english'))

In [3]:
notebook_dir = os.getcwd()
sop_df = pd.read_csv('../data/interim/sop_types_valid.csv', 
                     converters = {'juri': eval, 
                                   'filename': eval})
# sop_df.head(3)

In [4]:
# type_list = sop_df['type']

In [5]:
def load_event(event_type):
    with open(F'../data/sop_jsons/{event_type}.txt') as f:
        dct = json.load(f)
    f.close()
    return dct

In [6]:
# event_type = 'ANIMAL'
# dct = load_event(event_type)

In [7]:
# event_row = sop_df[sop_df['type'] == event_type]
# juri_to_filename = dict(zip(event_row['juri'].values[0], 
#                             event_row['filename'].values[0]))

In [8]:
# juris, roles, sops = list(), list(), list()
# for juri, role_sop in dct.items():
#     for role, sop in role_sop.items():
#         juris.append(juri)
#         roles.append(role)
#         sops.append(sop)
# df = pd.DataFrame({'juri': juris, 'role': roles, 'sop': sops})
# df['filename'] = df['juri'].apply(lambda x: juri_to_filename[x])

In [9]:
# df[ (df['role'] == 'dispatcher') & (df['juri'] == "RI") ]['sop'].values.tolist()

In [10]:
# df_calltaker = df[df['role'] == 'call taker']
# df_dispatcher = df[df['role'] == 'dispatcher']

In [11]:
# remove = [
#     "subject:",
#     "writes:",
#     "wrote:",
#     "write:"
# ]
# def multiple_replace(text, unwanted = remove):
#     """
#     Replace unwanted pattern with a space; return a string
#     """ 
#     reg = re.compile(f"({'|'.join(remove)})")
#     return reg.sub(" ", text)
# "(%s)" % "|".join(remove) == f"({'|'.join(remove)})"

In [12]:
def preprocess(strlist,
               min_token_len = 2,
               allowed_pos = ['ADV', 'ADJ', 'VERB', 'NOUN']): 
#     texts = ' '.join(strlist).lower()
    res = list()
    for string in strlist:
        text = string.lower()
        doc = nlp(text)
        res += [token.lemma_ for token in doc \
               if token.is_alpha \
               # Spacy considers 'call' as a stop word, which is not suitable for our case
               and not token.is_stop \
#                and token.text not in stop_words \              
               and token.pos_ in allowed_pos \
               and len(token.lemma_) > min_token_len]
    
    return ' '.join(res)

In [13]:
def get_dct_dtmatrix(sops):
    corpus = [sop.split() for sop in map(preprocess, sops)]
    dictionary = corpora.Dictionary(corpus)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]
    return doc_term_matrix, corpus, dictionary

In [14]:
# doc_term_matrix, corpus, dictionary = get_dct_dtmatrix(df_dispatcher['sop'])

In [15]:
def topics_with_coherence(doc_term_matrix, corpus, dictionary, texts, 
                          N = 10, passes = 20, coherence = 'c_v', random_state = 2020):
    num_topic, ldas, scores = list(), list(), list()
#     doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]
    for n in range(1, N+1):
        lda = models.LdaModel(corpus = doc_term_matrix, 
                              id2word = dictionary, 
                              num_topics = n, 
                              passes = passes, 
                              random_state = random_state)
        coherence_model = CoherenceModel(
            model = lda, 
            texts = corpus, 
            dictionary = dictionary, 
            coherence = coherence)
        coherence_score = coherence_model.get_coherence()
#         mod = {'num_topic': n, 'model': lda, 'coherence_score': coherence_score}
#         res[f"num_topic={n}"] = [lda, coherence_score]
        num_topic.append(n)
        ldas.append(lda)
        scores.append(coherence_score)
    return pd.DataFrame({
        'num_topic':num_topic,
        'model': ldas,
        'coherence_score': scores
    })

In [16]:
# call_coherence_cv = topics_with_coherence(doc_term_matrix, corpus, dictionary, 
#                                           df_dispatcher['sop'].values.tolist())
# call_coherence_cv

In [17]:
# fig, ax = plt.subplots(1, 1, figsize = (12, 8))
# ax.plot(call_coherence_cv.loc[:, 'num_topic'].values, 
#         call_coherence_cv.loc[:, 'coherence_score'].values)
# ax.set_xlabel('number of topics')
# ax.set_ylabel('coherence score')
# ax.set_title('Coherence Score (c_v) vs Number of Topics')
# ax.grid()
# plt.show()

In [18]:
# call_model_cv = call_coherence_cv.iloc[1:, :].sort_values('coherence_score')['model'].tolist()[-1]
# print(call_model_cv)

In [19]:
# call_model_cv.print_topics()

In [20]:
def get_topic(model, doc):
    ppdoc = preprocess(doc)
    doc_term_arr = dictionary.doc2bow(ppdoc.split())
    return sorted(model[doc_term_arr], 
                  key = lambda x: x[1], 
                  reverse = True)[0][0]

In [21]:
# df_call_withtopic = df_dispatcher.copy()
# df_call_withtopic.loc[:, 'topic_id'] = list(map(lambda x: get_topic(call_model_cv, x), 
#                                                 df_calltaker['sop'].values.tolist()))
# df_call_withtopic = df_call_withtopic.sort_values(by = ['topic_id', 'juri'], ignore_index = True)

In [22]:
# df_call_withtopic

In [23]:
# empty = pd.DataFrame()
# df1 = pd.DataFrame({'type': ['type1', 'type2'], 'value': [1, 2]})
# empty = empty.append(df1)
# empty = empty.append(df1)
# empty

#### Reflection of DRUGS coherence score
- the coherence score is very high for the one-topic model
- this makes sense, because we are looking at docs under the same type "DRUGS"

#### Question
- While the model assigns the documents with the correct topic, does this necessarily mean the documents are similar enough to be consolicated?
- LDA in not stable.  How will this instability affect us?

In [28]:
type_list = sop_df['type'].values.tolist()
type_list[0]

'1033'

In [29]:
type_list = sop_df['type']
res = pd.DataFrame()
for event_type in type_list:
    dct = load_event(event_type)
    event_row = sop_df[sop_df['type'] == event_type]
    juri_to_filename = dict(zip(event_row['juri'].values[0], 
                            event_row['filename'].values[0]))
    juris, roles, sops, types = list(), list(), list(), list()
    for juri, role_sop in dct.items():
        for role, sop in role_sop.items():
            juris.append(juri)
            roles.append(role)
            sops.append(sop)
            types.append(event_type)
    typedf = pd.DataFrame({'type': types, 'juri': juris, 'role': roles, 'sop': sops})
    typedf['filename'] = typedf['juri'].apply(lambda x: juri_to_filename[x])
    df_calltaker = typedf[typedf['role'] == 'call taker']
    df_dispatcher = typedf[typedf['role'] == 'dispatcher']
    print(df_calltaker.shape)
    print(df_dispatcher.shape)

    for df in [df_calltaker, df_dispatcher]:
        if len(df) == 0:
            continue
        print('Start working on:', event_type, df['role'].unique())
        doc_term_matrix, corpus, dictionary = get_dct_dtmatrix(df['sop'])
        coherence_cv = topics_with_coherence(doc_term_matrix, corpus, dictionary, 
                                            df['sop'].values.tolist())
        best_model_cv = coherence_cv.iloc[1:, :].sort_values('coherence_score')['model'].tolist()[-1]
        df_with_topic = df.copy()
        df_with_topic.loc[:, 'topic_id'] = list(map(lambda x: get_topic(best_model_cv, x), 
                                                        df['sop'].values.tolist()))
        df_with_topic = df_with_topic.sort_values(by = ['topic_id', 'juri'], ignore_index = True)
        res = res.append(df_with_topic)
        print('Finish working on:', event_type, df['role'].unique())

(16, 5)
(16, 5)
Start working on: 1033 ['call taker']
Finish working on: 1033 ['call taker']
Start working on: 1033 ['dispatcher']
Finish working on: 1033 ['dispatcher']
(16, 5)
(15, 5)
Start working on: DOMI ['call taker']
Finish working on: DOMI ['call taker']
Start working on: DOMI ['dispatcher']
Finish working on: DOMI ['dispatcher']
(15, 5)
(16, 5)
Start working on: FOUNDP ['call taker']
Finish working on: FOUNDP ['call taker']
Start working on: FOUNDP ['dispatcher']
Finish working on: FOUNDP ['dispatcher']
(16, 5)
(16, 5)
Start working on: HAZARD ['call taker']
Finish working on: HAZARD ['call taker']
Start working on: HAZARD ['dispatcher']
Finish working on: HAZARD ['dispatcher']
(16, 5)
(16, 5)
Start working on: IMPAIR ['call taker']
Finish working on: IMPAIR ['call taker']
Start working on: IMPAIR ['dispatcher']
Finish working on: IMPAIR ['dispatcher']
(16, 5)
(16, 5)
Start working on: 911 ['call taker']
Finish working on: 911 ['call taker']
Start working on: 911 ['dispatcher'

Finish working on: MISSIC ['dispatcher']
(15, 5)
(15, 5)
Start working on: ASLTI ['call taker']
Finish working on: ASLTI ['call taker']
Start working on: ASLTI ['dispatcher']
Finish working on: ASLTI ['dispatcher']
(15, 5)
(15, 5)
Start working on: ASLTSX ['call taker']
Finish working on: ASLTSX ['call taker']
Start working on: ASLTSX ['dispatcher']
Finish working on: ASLTSX ['dispatcher']
(15, 5)
(15, 5)
Start working on: PROST ['call taker']
Finish working on: PROST ['call taker']
Start working on: PROST ['dispatcher']
Finish working on: PROST ['dispatcher']
(15, 5)
(15, 5)
Start working on: ALARMS ['call taker']
Finish working on: ALARMS ['call taker']
Start working on: ALARMS ['dispatcher']
Finish working on: ALARMS ['dispatcher']
(15, 5)
(15, 5)
Start working on: ASSGP ['call taker']
Finish working on: ASSGP ['call taker']
Start working on: ASSGP ['dispatcher']
Finish working on: ASSGP ['dispatcher']
(15, 5)
(15, 5)
Start working on: THEFT ['call taker']
Finish working on: THEFT [

Finish working on: SPAT ['call taker']
Start working on: SPAT ['dispatcher']
Finish working on: SPAT ['dispatcher']
(14, 5)
(14, 5)
Start working on: ALARMD ['call taker']
Finish working on: ALARMD ['call taker']
Start working on: ALARMD ['dispatcher']
Finish working on: ALARMD ['dispatcher']
(14, 5)
(14, 5)
Start working on: DNA ['call taker']
Finish working on: DNA ['call taker']
Start working on: DNA ['dispatcher']
Finish working on: DNA ['dispatcher']


In [30]:
res

,type,juri,role,sop,filename,topic_id
0,1033,AB,call taker,"[Create a call, Questions, Are there weapons i...",AB - 1033 - Officer in trouble.docx,7
1,1033,BI,call taker,"[Create a call, Questions, Are there weapons i...",BI - 1033 - Officer in trouble.docx,7
2,1033,BU,call taker,"[Create a call, Questions, Are there weapons i...",BU - 1033 - Officer in trouble.docx,7
3,1033,DE,call taker,"[Create a call, Questions, Are there weapons i...",DE - 1033 - Officer in trouble.docx,7
4,1033,DFPF,call taker,"[Create a call, Questions, Are there weapons i...",DFPF - 1033 - Officer in trouble.docx,7
...,...,...,...,...,...,...
9,DNA,WV,dispatcher,[This file type is not used. ],WV - DNA - DNA collection.docx,1
10,DNA,NW,dispatcher,[Person attends front counter or member reques...,NW - DNA - DNA collection .docx,3
11,DNA,PO,dispatcher,[Assign a member],PO - DNA - DNA collection.docx,3
12,DNA,RI,dispatcher,"[Treat file as a Priority 2, Assign a DNA cert...",RI - DNA - DNA collection.docx,3


In [31]:
ress = res.reset_index(drop = True)
ress

,type,juri,role,sop,filename,topic_id
0,1033,AB,call taker,"[Create a call, Questions, Are there weapons i...",AB - 1033 - Officer in trouble.docx,7
1,1033,BI,call taker,"[Create a call, Questions, Are there weapons i...",BI - 1033 - Officer in trouble.docx,7
2,1033,BU,call taker,"[Create a call, Questions, Are there weapons i...",BU - 1033 - Officer in trouble.docx,7
3,1033,DE,call taker,"[Create a call, Questions, Are there weapons i...",DE - 1033 - Officer in trouble.docx,7
4,1033,DFPF,call taker,"[Create a call, Questions, Are there weapons i...",DFPF - 1033 - Officer in trouble.docx,7
...,...,...,...,...,...,...
2924,DNA,WV,dispatcher,[This file type is not used. ],WV - DNA - DNA collection.docx,1
2925,DNA,NW,dispatcher,[Person attends front counter or member reques...,NW - DNA - DNA collection .docx,3
2926,DNA,PO,dispatcher,[Assign a member],PO - DNA - DNA collection.docx,3
2927,DNA,RI,dispatcher,"[Treat file as a Priority 2, Assign a DNA cert...",RI - DNA - DNA collection.docx,3


In [32]:
from datetime import datetime 
dt = datetime.now().strftime('%Y-%m-%dT%H_%M_%S')
cwd = os.getcwd()
os.chdir(notebook_dir)
ress.to_csv(f'../data/interim/sop_topics_{dt}.csv', index = False)
os.chdir(cwd)

In [33]:
print(type_list.values.tolist())

['1033', 'DOMI', 'FOUNDP', 'HAZARD', 'IMPAIR', '911', 'INTELL', 'JUMPER', 'MAND', 'MISCH', 'MISCHI', 'MVI', 'MVIHR', 'MVIINJ', 'OCC', 'PANHA', 'PROP', 'PURSUE', 'RECVEH', 'SCREAM', 'SHOTS', 'SIP', 'SUDDEN', 'SUICID', 'SUSPC', 'SUSPP', 'THEFTV', 'THREATS', 'TRAFF', 'TRAFFS', 'WEAPON', 'DRUGS', 'INSEC', 'ANIMAL', 'ASSOA', 'ABANDV', 'ARREST', 'ARSON', 'MVIPOL', 'PARK', 'ANNOY', 'OVERD', 'NOK', 'BREACH', 'PROWL', 'ASLT', 'MISSIP', 'MISSIC', 'ASLTI', 'ASLTSX', 'PROST', 'ALARMS', 'ASSGP', 'THEFT', 'AB911 CELL', 'WARRAN', 'UNWANT', 'AB911 LANDLINE', 'ABDUC', 'THEFTI', 'SUSPV', 'ROBB', 'AIREM', 'ALARM', 'STALK', 'SHOPL', 'ALARMH', 'ROBBI', 'MARINE', 'YOUTH', 'LIQUOR', 'COUNTERFEIT', 'EXPLOS', 'EXTORT', 'BNEI', 'DOMRPT', 'FIGHT', 'FIREAR', 'BOMB', 'BNE', 'FRAUD', 'DISTB', 'HARASS', 'DEMON', 'ASSPFA', 'HOMEIN', 'HOSTAG', 'BAIT', 'CHECK', 'BYLAW', 'INDEC', 'KPEACE', 'INDUST', 'ASSMHA', 'SPAT', 'ALARMD', 'DNA']


In [34]:
ress[ (ress['type'] == 'MISCH') & (ress['role'] == 'call taker')]

,type,juri,role,sop,filename,topic_id
285,MISCH,DFPF,call taker,[Refer caller to Police of Jurisdiction],DFPF - MISCH - Mischief.docx,0
286,MISCH,VA,call taker,[If the mischief involves any of the following...,VA - MISCH - Mischief.docx,0
287,MISCH,BI,call taker,"[Create a call, Mischief to auto, Create a cal...",BI - MISCH - Mischief.docx,2
288,MISCH,DE,call taker,"[Create a call, Mischief to auto, Create a cal...",DE - MISCH - Mischief.docx,2
289,MISCH,NW,call taker,"[Create a call, Mischief to auto, Create a cal...",NW - MISCH - Mischief.docx,2
290,MISCH,PO,call taker,"[Create a call, Mischief to auto, Create a cal...",PO - MISCH - Mischief.docx,2
291,MISCH,RM,call taker,"[Create a call, Mischief to auto, Create a cal...",RM - MISCH - Mischief.docx,2
292,MISCH,SC,call taker,"[Create a call, Mischief to auto, Create a cal...",SC - MISCH - Mischief.docx,2
293,MISCH,SQ,call taker,"[Create a call, Mischief to auto, Create a cal...",SQ - MISCH - Mischief.docx,2
294,MISCH,SX,call taker,"[Create a call, Firecracker / Fireworks use , ...",SX - MISCH - Mischief.docx,2


In [30]:
ress[ (ress['type'] == 'MISCH') & (ress['role'] == 'dispatcher')]

,type,juri,role,sop,filename,topic_id
301,MISCH,AB,dispatcher,[Property (includes requeued calls and value >...,AB - MISCH - Mischief.docx,1
302,MISCH,BU,dispatcher,"[Property (includes known suspect, value > $50...",BU - MISCH - Mischief.docx,1
303,MISCH,RI,dispatcher,"[Firecracker / Fireworks use, General broadcas...",RI - MISCH - Mischief .docx,1
304,MISCH,SX,dispatcher,"[If a member is on shift, Assign a member, If ...",SX - MISCH - Mischief.docx,1
305,MISCH,BI,dispatcher,"[Assign a member, Mischief to auto, Assign a m...",BI - MISCH - Mischief.docx,2
306,MISCH,DE,dispatcher,"[Assign a member, Mischief to auto, Assign a m...",DE - MISCH - Mischief.docx,2
307,MISCH,NW,dispatcher,"[Assign a member, Mischief to auto, Assign a m...",NW - MISCH - Mischief.docx,2
308,MISCH,PO,dispatcher,"[Assign a member, Mischief to auto, Assign a m...",PO - MISCH - Mischief.docx,2
309,MISCH,RM,dispatcher,[Assign a member],RM - MISCH - Mischief.docx,2
310,MISCH,SC,dispatcher,"[Assign a member, Mischief to auto, Assign a m...",SC - MISCH - Mischief.docx,2


In [31]:
ress[ (ress['type'] == 'ANIMAL') & (ress['role'] == 'call taker')]

,type,juri,role,sop,filename,topic_id
1027,ANIMAL,BI,call taker,"[Animal bites, Create a call, Animal left in a...",BI - ANIMAL.docx,0
1028,ANIMAL,BU,call taker,"[Animal bites, Create a call, Animal left in a...",BU - ANIMAL.docx,0
1029,ANIMAL,DFPF,call taker,"[Dogs at large, Dog is running loose and poses...",DFPF - ANIMAL.docx,0
1030,ANIMAL,PO,call taker,"[Animal bites, Create a call if any of the fol...",PO - ANIMAL.docx,0
1031,ANIMAL,RI,call taker,"[Animal bites, Create a call, Animal left in a...",RI - ANIMAL.docx,0
1032,ANIMAL,RM,call taker,"[Animal bites, Create a call, Animal left in a...",RM - ANIMAL.docx,0
1033,ANIMAL,SC,call taker,"[Animal bites, Create a call, Animal left in a...",SC - ANIMAL.docx,0
1034,ANIMAL,SQ,call taker,"[Animal bites, Create a call, Animal left in a...",SQ - ANIMAL.docx,0
1035,ANIMAL,SX,call taker,"[Animal bites, Create a call, Animal left in a...",SX - ANIMAL.docx,0
1036,ANIMAL,UN,call taker,"[Animal bites, Create a call, Animal left in a...",UN - ANIMAL.docx,0


In [32]:
ress[ (ress['type'] == 'DRUGS') & (ress['role'] == 'call taker')]

,type,juri,role,sop,filename,topic_id
964,DRUGS,AB,call taker,"[Persons congregating and dealing/using drugs,...",AB - DRUGS.docx,1
965,DRUGS,BI,call taker,"[Persons congregating and dealing/using Drugs,...",BI - DRUGS.docx,1
966,DRUGS,BU,call taker,"[Persons congregating and dealing/using Drugs,...",BU - DRUGS.docx,1
967,DRUGS,DE,call taker,"[Persons congregating and dealing/using Drugs,...",DE - DRUGS.docx,1
968,DRUGS,NW,call taker,"[Persons congregating and dealing/using Drugs,...",NW - DRUGS.docx,1
969,DRUGS,PO,call taker,"[Persons congregating and dealing/using drugs,...",PO - DRUGS.docx,1
970,DRUGS,RI,call taker,"[Persons congregating and dealing/using Drugs,...",RI - DRUGS.docx,1
971,DRUGS,RM,call taker,"[Persons congregating and dealing/using Drugs,...",RM - DRUGS.docx,1
972,DRUGS,SC,call taker,"[Persons congregating and dealing/using Drugs,...",SC - DRUGS.docx,1
973,DRUGS,SQ,call taker,"[Persons congregating and dealing/using Drugs,...",SQ - DRUGS.docx,1


In [39]:
ress['sop'] = ress['sop'].apply(lambda x: ' '.join(x))

In [40]:
ress.sample(5)

,type,juri,role,sop,filename,topic_id
2432,FRAUD,PO,call taker,In Progress – Suspect is on scene Create a cal...,PO - FRAUD.docx,0
505,PROP,AB,call taker,Located international passport Caller wants to...,"AB - PROP - Property lost, found or seized.docx",2
2310,FIGHT,BI,call taker,Create a call Questions Are weapons involved? ...,BI - FIGHT - Fight.docx,1
1438,MISSIP,WP,dispatcher,Assign a member. Advise field supervisor / Dut...,WP - MISSIP - Missing person.docx,3
2679,CHECK,UN,call taker,Caller is usually in regular contact: Create a...,UN - CHECK - Check wellbeing.docx,2


In [41]:
ress.to_csv('../data/sop.csv', index=False)

In [33]:
ress[ (ress['type'] == 'DRUGS') & (ress['role'] == 'call taker')]['sop'].values.tolist()[0]

['Persons congregating and dealing/using drugs',
 'Create a call',
 'Obtain:',
 'Full descriptions of party/parties involved',
 'Location where their drugs are being hidden if known',
 'If caller wants to remain anonymous enter that in remarks of call',
 'Meth Labs',
 'Create a call',
 'Questions:',
 'What evidence suggests the location in question is a Grow or Meth Lab? i.e. smells, window coverings 24/7, condensation on the windows, humming noise from a generator, no snow on roof (based on time of year)',
 'Does the caller know who owns/rents the location',
 'Physical description of suspects?',
 'Are there any suspects/occupants at the location now?',
 'Do people/vehicles attend at odd times of day?',
 'Are there any associated vehicle descriptions and/or license plates?',
 'Are there any cameras or motion/timer lights visible?',
 'Is the caller willing to speak to an officer about their concerns?',
 'The caller is permitted to be anonymous. If they wish to remain anonymous explain t

In [34]:
# all_coherence = topics_with_coherence(dt_matrix_all, corpus_all, dictionary_all, N = 20)

In [35]:
# all_coherence

In [36]:
# plt.figure(figsize = (12, 8))
# plt.plot(all_coherence.loc[:, 'num_topic'].values, all_coherence.loc[:, 'coherence_score'].values)
# plt.show()